In [126]:
import numpy as np
from utils import *
from train_utils import *

# Introduction

This is a continuation of the previous project for classifying MNIST digits, this time implementing a neural network. Problem statements from edX are italicized.

# Activation Functions

*The first step is to design the activation function for each neuron. In this problem, we will initialize the network weights to 1, use **ReLU** for the activation function of the hidden layers, and use an identity function for the output neuron. The hidden layer has a bias but the output layer does not. Complete the helper functions in `neural_networks.py`, including `rectified_linear_unit` and `rectified_linear_unit_derivative`, for you to use in the `NeuralNetwork` class, and implement them below.*

In [27]:
def rectified_linear_unit(x):
    """ Returns the ReLU of x, or the maximum between 0 and x."""
    return np.max([x,0])

## Taking the Derivative

Now implement its derivative so that we can properly run backpropagation when training the net. Note: we will consider the derivative at zero to have the same value as the derivative at all negative points.

Since
\begin{equation}
  ReLU(x) =
  \begin{cases}
                                   x & \text{if $x > 0$} \\
                                   0 & \text{if $x \leq 0$}
  \end{cases}
\end{equation}

Then
\begin{equation}
  \frac{d}{dx}ReLU(x) =
  \begin{cases}
                                   1 & \text{if $x > 0$} \\
                                   0 & \text{if $x \leq 0$}
  \end{cases}
\end{equation}

This can be implemented easily with an `if` statement.

In [28]:
def rectified_linear_unit_derivative(x):
    """ Returns the ReLU of x, or the maximum between 0 and x."""
    if x <= 0:
        return 0
    else:
        return 1

# Training the Network

*Forward propagation is simply the summation of the previous layer's output multiplied by the weight of each wire, while back-propagation works by computing the partial derivatives of the cost function with respect to every weight or bias in the network. In back propagation, the network gets better at minimizing the error and predicting the output of the data being used for training by incrementally updating their weights and biases using stochastic gradient descent.*

*We are trying to estimate a continuous-valued function, thus we will use squared loss as our cost function and an identity function as the output activation function.  $f(x)$  is the activation function that is called on the input to our final layer output node, and  $\hat a$  is the predicted value, while  y  is the actual value of the input.*

\begin{align}
   C &= \frac{1}{2}(y−a)^2 \\
f(x) &= x
\end{align} 


*When you're done implementing the function `train` (below and in your local repository), run the script and see if the errors are decreasing. If your errors are all under 0.15 after the last training iteration then you have implemented the neural network training correctly.*

*You'll notice that the train function inherits from `NeuralNetworkBase` in the codebox below; this is done for grading purposes. In your local code, you implement the function directly in your Neural Network class all in one file. The rest of the code in `NeuralNetworkBase` is the same as in the original `NeuralNetwork` class you have locally.*

In [ ]:
class NeuralNetwork(NeuralNetworkBase):

    def train(self, x1, x2, y):

        # Use standard variables and store as arrays
        W1 = np.array(self.input_to_hidden_weights) # 3 x 2
        b = np.array(self.biases) # 3 x 1
        W2 = self.hidden_to_output_weights #np.array(self.hidden_to_output_weights) # 1 x 3
        f2 = output_layer_activation
        df2 = output_layer_activation_derivative
        eta = self.learning_rate

        # Vectorize ReLU functions
        f1 = np.vectorize(rectified_linear_unit)
        df1 = np.vectorize(rectified_linear_unit_derivative)

        ### Forward propagation ###
        x = np.array([[x1], [x2]]) # 2 x 1

        # Calculate the input and activation of the hidden layer
        z1 = W1 @ x + b # 3 x 1
        h1 = f1(z1) # 3 x 1

        z2 =  W2 @ h1 
        o1 = f2(z2)

        ### Backpropagation ###
        # Compute gradients
        bias_gradients = np.multiply( df1(z1), np.transpose(W2) ) * (o1 - y)
        input_to_hidden_weight_gradients = np.multiply(np.vstack((x.T, x.T, x.T)), np.hstack((bias_gradients, bias_gradients)))
        hidden_to_output_weight_gradients = np.transpose( np.multiply(h1, (o1 - y)) )

        # Use gradients to adjust weights and biases using gradient descent
        self.biases = b - eta * bias_gradients
        self.input_to_hidden_weights = W1 - eta * input_to_hidden_weight_gradients
        self.hidden_to_output_weights = W2 - eta * hidden_to_output_weight_gradients

# Predicting the Test Data

*Now fill in the code for the function `predict`, which will use your trained neural network in order to label new data.*

In [ ]:
class NeuralNetwork(NeuralNetworkBase):

    def predict(self, x1, x2):

       # Use standard variables and store as arrays
        W1 = np.array(self.input_to_hidden_weights)
        b = np.array(self.biases)
        W2 = np.array(self.hidden_to_output_weights)
        f2 = output_layer_activation
        df2 = output_layer_activation_derivative
        eta = self.learning_rate
        
        # Vectorize ReLU functions
        f1 = np.vectorize(rectified_linear_unit)
        df1 = np.vectorize(rectified_linear_unit_derivative)
        
        ### Forward propagation ###
        x = np.array([[x1], [x2]])
        
        # Compute output for a single input(should be same as the forward propagation in training)
        z1 = W1 @ x + b
        h1 = f1(z1)
        z2 = W2 @ h1
        o1 = f2(z2)

        return o1.item()

This block of code is simply the chunk from the train() function above for forward propagation.

## Testing the Network

In [175]:
%run -i neural_nets

Point, (1, 1) Prediction, [7.0384532]
Test Passed
Point, (2, 2) Prediction, [14.04281483]
Test Passed
Point, (3, 3) Prediction, [21.04717646]
Test Passed
Point, (5, 5) Prediction, [35.05589972]
Test Passed
Point, (10, 10) Prediction, [70.07770787]
Test Passed


# Conceptual Questions

1. *What is the danger to having too many hidden units in your network?*
    - It will take up more memory
    - It may overfit the training data
    - It will take longer to train

2. *What would happen if you run the code for more epochs in terms of training and testing accuracy? What should we expect to see?*
    - As above, the network would take longer to train. More importantly, it could potentially overfit to the training data, which would likely cause it to perform poorly on unseen data.
    
![Accuracy vs. Epochs](acc-epoch.png)

# Classification for MNIST Using Deep Neural Networks
*In this section, we are going to use deep neural networks to perform the same classification task as in previous sections. We will use PyTorch, a python deep learning framework. Using a framework like PyTorch means you don't have to implement all of the details (like in the earlier problem) and can spend more time thinking through your high level architecture.*

# Fully-Connected Neural Networks

*First, we will employ the most basic form of a deep neural network, in which the neurons in adjacent layers are fully connected to one another.*

## Training and Testing Accuracy Over Time

*We have provided a toy example **nnet_fc.py** in which we have implemented for you a simple neural network. This network has one hidden layer of 10 neurons with a rectified linear unit (ReLU) nonlinearity, as well as an output layer of 10 neurons (one for each digit class). Finally, a softmax function normalizes the activations of the output neurons so that they specify a probability distribution. Reference the PyTorch Documentation and read through it in order to gain a better understanding of the code. Then, try running the code on your computer with the command `Python3 nnet_fc.py`. This will train the network with 10 epochs, where an epoch is a complete pass through the training dataset. Total training time of your network should take no more than a couple of minutes. At the end of training, your model should have an accuracy of more than  $85\%$  on test data.*

In [11]:
def NN(batch_size=32, lr=0.1, momentum=0, hidden_units=10):
    np.random.seed(12321)  # for reproducibility
    torch.manual_seed(12321)  # for reproducibility
    
    # Load the dataset
    num_classes = 10
    X_train, y_train, X_test, y_test = get_MNIST_data()

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = y_train[dev_split_index:]
    X_train = X_train[:dev_split_index]
    y_train = y_train[:dev_split_index]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [y_train[i] for i in permutation]

    # Split dataset into batches
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    #################################
    ## Model specification - BASELINE
    model = nn.Sequential(
              nn.Linear(784, hidden_units), # in_features = 28x28 (pixels in image), out_features = 10 (neurons in hidden layer)
              nn.ReLU(), # Hidden layer activation
              nn.Linear(hidden_units, 10), # 10 inputs from previous layer, 10 outputs (one for each digit)
            )
    ##################################

    train_model(train_batches, dev_batches, model, lr=lr, momentum=momentum)

    ## Evaluate the model on test data
    loss, accuracy = run_epoch(test_batches, model.eval(), None)

    print("-------------\nLoss on test set:"  + str(loss) + "\nAccuracy on test set: " + str(accuracy))

In [12]:
NN()

-------------
Epoch 1:



100%|████████████████████████████████████████████████████████████████████████████| 1687/1687 [00:00<00:00, 1939.10it/s]


Train loss: 0.445684 | Train accuracy: 0.870758


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 4573.09it/s]


Val loss:   0.260342 | Val accuracy:   0.923630
-------------
Epoch 2:



...

Train loss: 0.223966 | Train accuracy: 0.935147


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 4807.65it/s]


Val loss:   0.228388 | Val accuracy:   0.932487


100%|██████████████████████████████████████████████████████████████████████████████| 312/312 [00:00<00:00, 4469.13it/s]


-------------
Loss on test set:0.2672268722492915
Accuracy on test set: 0.9204727564102564


## Improving Accuracy - Modifying Hyper Parameters

*We would like to try to improve the performance of the model by performing a mini grid search over hyper parameters (note that a full grid search should include more values and combinations).*

*To this end, we will use our **baseline model** (batch size 32, hidden size 10, learning rate 0.1, momentum 0 and the ReLU activation function) and modify one parameter each time while keeping all others to the baseline.*

*We will use the validation accuracy of the model after training for 10 epochs. For the LeakyReLU activation function, use the default parameters from pyTorch (negative_slope=0.01).*

### Batch Size
Batch size increased from 32 to 64.

In [6]:
NN(batch_size=64)

-------------
Epoch 1:



100%|██████████████████████████████████████████████████████████████████████████████| 843/843 [00:00<00:00, 1214.45it/s]


Train loss: 0.528406 | Train accuracy: 0.851016


100%|████████████████████████████████████████████████████████████████████████████████| 93/93 [00:00<00:00, 2825.90it/s]


Val loss:   0.270610 | Val accuracy:   0.922715
-------------
Epoch 2:



...

Train loss: 0.225577 | Train accuracy: 0.934794


100%|████████████████████████████████████████████████████████████████████████████████| 93/93 [00:00<00:00, 2914.08it/s]


Val loss:   0.208171 | Val accuracy:   0.940020


100%|██████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 3067.00it/s]


-------------
Loss on test set:0.24238464534760285
Accuracy on test set: 0.9314903846153846


### Learning Rate
Learning rate decreased by a factor of 10 (`lr = 0.01`).

In [7]:
NN(lr=0.01)

-------------
Epoch 1:



100%|████████████████████████████████████████████████████████████████████████████| 1687/1687 [00:00<00:00, 1967.81it/s]


Train loss: 1.060107 | Train accuracy: 0.720417


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 4573.17it/s]


Val loss:   0.422035 | Val accuracy:   0.895388
-------------
Epoch 2:



...

Train loss: 0.285981 | Train accuracy: 0.918179


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 4261.39it/s]


Val loss:   0.233266 | Val accuracy:   0.934659


100%|██████████████████████████████████████████████████████████████████████████████| 312/312 [00:00<00:00, 4010.70it/s]


-------------
Loss on test set:0.2788660805194806
Accuracy on test set: 0.9206730769230769


### Momentum
Momentum increased to `0.9`.

In [8]:
NN(momentum=0.9)

-------------
Epoch 1:



100%|████████████████████████████████████████████████████████████████████████████| 1687/1687 [00:01<00:00, 1579.50it/s]


Train loss: 0.612039 | Train accuracy: 0.823614


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 4464.29it/s]


Val loss:   0.433070 | Val accuracy:   0.884358
-------------
Epoch 2:



...

Train loss: 0.386717 | Train accuracy: 0.905379


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 4360.57it/s]


Val loss:   0.371283 | Val accuracy:   0.913102


100%|██████████████████████████████████████████████████████████████████████████████| 312/312 [00:00<00:00, 6133.95it/s]


-------------
Loss on test set:0.4073197087750603
Accuracy on test set: 0.9078525641025641


### LeakyReLU Activation
Modify the hidden layer activation function.

In [13]:
def NN_Leaky(batch_size=32, lr=0.1, momentum=0, hidden_units=10):
    np.random.seed(12321)  # for reproducibility
    torch.manual_seed(12321)  # for reproducibility
    
    # Load the dataset
    num_classes = 10
    X_train, y_train, X_test, y_test = get_MNIST_data()

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = y_train[dev_split_index:]
    X_train = X_train[:dev_split_index]
    y_train = y_train[:dev_split_index]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [y_train[i] for i in permutation]

    # Split dataset into batches
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    #################################
    ## Model specification - BASELINE
    model = nn.Sequential(
              nn.Linear(784, hidden_units), # in_features = 28x28 (pixels in image), out_features = 10 (neurons in hidden layer)
              nn.LeakyReLU(), # Hidden layer activation
              nn.Linear(hidden_units, 10), # 10 inputs from previous layer, 10 outputs (one for each digit)
            )
    ##################################

    train_model(train_batches, dev_batches, model, lr=lr, momentum=momentum)

    ## Evaluate the model on test data
    loss, accuracy = run_epoch(test_batches, model.eval(), None)

    print("-------------\nLoss on test set:"  + str(loss) + "\nAccuracy on test set: " + str(accuracy))

In [10]:
NN_Leaky()

-------------
Epoch 1:



100%|████████████████████████████████████████████████████████████████████████████| 1687/1687 [00:00<00:00, 1894.20it/s]


Train loss: 0.444938 | Train accuracy: 0.871128


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 3989.37it/s]


Val loss:   0.261497 | Val accuracy:   0.922627
-------------
Epoch 2:



...

Train loss: 0.224515 | Train accuracy: 0.935147


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 4261.30it/s]


Val loss:   0.227615 | Val accuracy:   0.931985


100%|██████████████████████████████████████████████████████████████████████████████| 312/312 [00:00<00:00, 3769.10it/s]


-------------
Loss on test set:0.2689260167236894
Accuracy on test set: 0.9207732371794872


### Summary

<table>
  <tr>
    <th>Modification</th>
    <th>Validation Accuracy</th>
    <th>Test Accuracy</th>
  </tr>
  <tr>
    <td>Baseline</td>
    <td>0.932487</td>
    <td>0.920472</td>
  </tr>
  <tr>
    <td>Batch Size</td>
    <td>0.940020</td>
    <td>0.931490</td>
  </tr>
  <tr>
    <td>Learning Rate</td>
    <td>0.934659</td>
    <td>0.920673</td>
  </tr>
  <tr>
    <td>Momentum</td>
    <td>0.913102</td>
    <td>0.907852</td>
  </tr>
  <tr>
    <td>LeakyReLU</td>
    <td>0.931985</td>
    <td>0.920773</td>
  </tr>

</table>

As summarized in the table above, increasing the batch size resulted in the best performance in terms of validation and test accuracy.

## Improving Accuracy - Modifying Architecture
*Modifying the model's architecture is also worth considering. Increase the hidden representation size from 10 to 128 and repeat the grid search over the same hyper parameters.*

### Baseline

In [14]:
NN(hidden_units=128)

-------------
Epoch 1:



100%|█████████████████████████████████████████████████████████████████████████████| 1687/1687 [00:01<00:00, 976.61it/s]


Train loss: 0.366301 | Train accuracy: 0.897025


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 2640.83it/s]


Val loss:   0.177677 | Val accuracy:   0.948028
-------------
Epoch 2:



...

Train loss: 0.034583 | Train accuracy: 0.991294


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 2678.34it/s]


Val loss:   0.072713 | Val accuracy:   0.977941


100%|██████████████████████████████████████████████████████████████████████████████| 312/312 [00:00<00:00, 2673.79it/s]


-------------
Loss on test set:0.07395259204965371
Accuracy on test set: 0.9771634615384616


### Batch Size

In [18]:
NN(batch_size=64, hidden_units=128)

-------------
Epoch 1:



100%|███████████████████████████████████████████████████████████████████████████████| 843/843 [00:01<00:00, 827.06it/s]


Train loss: 0.463914 | Train accuracy: 0.875315


100%|████████████████████████████████████████████████████████████████████████████████| 93/93 [00:00<00:00, 1942.71it/s]


Val loss:   0.232220 | Val accuracy:   0.932460
-------------
Epoch 2:



...

Train loss: 0.065312 | Train accuracy: 0.982095


100%|████████████████████████████████████████████████████████████████████████████████| 93/93 [00:00<00:00, 2072.23it/s]


Val loss:   0.079208 | Val accuracy:   0.976815


100%|██████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 2031.37it/s]


-------------
Loss on test set:0.08362655630574012
Accuracy on test set: 0.9748597756410257


### Learning Rate

In [15]:
NN(lr=0.01, hidden_units=128)

-------------
Epoch 1:



100%|█████████████████████████████████████████████████████████████████████████████| 1687/1687 [00:01<00:00, 996.76it/s]


Train loss: 0.908129 | Train accuracy: 0.796792


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 2884.59it/s]


Val loss:   0.377350 | Val accuracy:   0.906250
-------------
Epoch 2:



...

Train loss: 0.208362 | Train accuracy: 0.941483


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 2678.57it/s]


Val loss:   0.169065 | Val accuracy:   0.955047


100%|██████████████████████████████████████████████████████████████████████████████| 312/312 [00:00<00:00, 2768.42it/s]


-------------
Loss on test set:0.19775969692720816
Accuracy on test set: 0.9427083333333334


### Momentum

In [16]:
NN(momentum=0.9, hidden_units=128)

-------------
Epoch 1:



100%|█████████████████████████████████████████████████████████████████████████████| 1687/1687 [00:01<00:00, 844.07it/s]


Train loss: 0.296532 | Train accuracy: 0.910325


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 2884.63it/s]


Val loss:   0.175828 | Val accuracy:   0.951370
-------------
Epoch 2:



...

Train loss: 0.082375 | Train accuracy: 0.979605


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 2976.26it/s]


Val loss:   0.189066 | Val accuracy:   0.963737


100%|██████████████████████████████████████████████████████████████████████████████| 312/312 [00:00<00:00, 2951.27it/s]


-------------
Loss on test set:0.24351653198783213
Accuracy on test set: 0.9580328525641025


### LeakyReLU Activation

In [17]:
NN_Leaky(hidden_units=128)

-------------
Epoch 1:



100%|████████████████████████████████████████████████████████████████████████████| 1687/1687 [00:01<00:00, 1031.01it/s]


Train loss: 0.366998 | Train accuracy: 0.897025


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 3073.83it/s]


Val loss:   0.179281 | Val accuracy:   0.947527
-------------
Epoch 2:



...

Train loss: 0.035277 | Train accuracy: 0.991108


100%|██████████████████████████████████████████████████████████████████████████████| 187/187 [00:00<00:00, 2840.95it/s]


Val loss:   0.070790 | Val accuracy:   0.978108


100%|██████████████████████████████████████████████████████████████████████████████| 312/312 [00:00<00:00, 2951.25it/s]


-------------
Loss on test set:0.07374823891008511
Accuracy on test set: 0.9776642628205128


### Summary

<table>
  <tr>
    <th>Modification</th>
    <th>Validation Accuracy</th>
    <th>Test Accuracy</th>
  </tr>
  <tr>
    <td>Baseline</td>
    <td>0.977941</td>
    <td>0.977163</td>
  </tr>
  <tr>
    <td>Batch Size</td>
    <td>0.976815</td>
    <td>0.974859</td>
  </tr>
  <tr>
    <td>Learning Rate</td>
    <td>0.955047</td>
    <td>0.942708</td>
  </tr>
  <tr>
    <td>Momentum</td>
    <td>0.963737</td>
    <td>0.958032</td>
  </tr>
  <tr>
    <td>LeakyReLU</td>
    <td>0.978108</td>
    <td>0.977664</td>
  </tr>

</table>

Increasing the neuron count in the hidden layer greatly improved the performance for each modification. Using a LeakyReLU activation resulted in the best performance.

# Convolutional Neural Networks

*Next, we are going to apply convolutional neural networks to the same task. These networks have demonstrated great performance on many deep learning tasks, especially in computer vision.*

*We provide skeleton code `part2-mnist/nnet_cnn.py` which includes examples of some (not all) of the new layers you will need in this part. Using the PyTorch Documentation, complete the code to implement a convolutional neural network with following layers in order:*

- A convolutional layer with 32 filters of size  3×3 

- A ReLU nonlinearity

- A max pooling layer with size  2×2 

- A convolutional layer with 64 filters of size  3×3 

- A ReLU nonlinearity

- A max pooling layer with size  2×2 

- A flatten layer

- A fully connected layer with 128 neurons

- A dropout layer with drop probability 0.5

- A fully-connected layer with 10 neurons

Note: We are not using a softmax layer because it is already present in the loss: PyTorch's `nn.CrossEntropyLoss` combines `nn.LogSoftMax` with `nn.NLLLoss`.

In [143]:
def CNN(batch_size=32, lr=0.1, momentum=0, hidden_units=128):
    np.random.seed(12321)  # for reproducibility
    torch.manual_seed(12321)  # for reproducibility
    
    # Load the dataset
    num_classes = 10
    X_train, y_train, X_test, y_test = get_MNIST_data()

    # Split into train and dev
    dev_split_index = int(9 * len(X_train) / 10)
    X_dev = X_train[dev_split_index:]
    y_dev = y_train[dev_split_index:]
    X_train = X_train[:dev_split_index]
    y_train = y_train[:dev_split_index]

    permutation = np.array([i for i in range(len(X_train))])
    np.random.shuffle(permutation)
    X_train = [X_train[i] for i in permutation]
    y_train = [y_train[i] for i in permutation]

    # Split dataset into batches
    train_batches = batchify_data(X_train, y_train, batch_size)
    dev_batches = batchify_data(X_dev, y_dev, batch_size)
    test_batches = batchify_data(X_test, y_test, batch_size)

    #################################
    ## Model specification - BASELINE
    model = nn.Sequential(
                # Convolution layer: in_channels, out_channels, kernel_size, stride
                nn.Conv2d(1, 32, (3, 3)),
                nn.ReLU(),
                nn.MaxPool2d((2, 2)),
                nn.Conv2d(32, 64, (3, 3)),
                nn.ReLU(),
                nn.MaxPool2d((2, 2)),
                Flatten(),
                nn.Linear(5 * 5 * 64, hidden_units),
                nn.Dropout(0.5),
                nn.Linear(hidden_units, 10)
            )
    ##################################

    train_model(train_batches, dev_batches, model, lr=lr, momentum=momentum)

    ## Evaluate the model on test data
    loss, accuracy = run_epoch(test_batches, model.eval(), None)

    print("-------------\nLoss on test set:"  + str(loss) + "\nAccuracy on test set: " + str(accuracy))

In [147]:
%run -i nnet_cnn

-------------
Epoch 1:



100%|██████████████████████████████████████████████████████████████████████████████| 1687/1687 [00:48<00:00, 34.73it/s]


Train loss: 0.244054 | Train accuracy: 0.923459


100%|███████████████████████████████████████████████████████████████████████████████| 187/187 [00:01<00:00, 159.44it/s]


Val loss:   0.059416 | Val accuracy:   0.983122
-------------
Epoch 2:



...

Train loss: 0.020331 | Train accuracy: 0.993405


100%|███████████████████████████████████████████████████████████████████████████████| 187/187 [00:01<00:00, 151.21it/s]


Val loss:   0.036694 | Val accuracy:   0.991143


100%|███████████████████████████████████████████████████████████████████████████████| 312/312 [00:01<00:00, 162.51it/s]


Loss on test set:0.02735500419751192 Accuracy on test set: 0.9904847756410257


The code above shows what is being run from **nnet_cnn.py**, while the following cell actually runs the file. 

Although it took some time to run all epochs (8 minutes 28 seconds) this model performs extremely well, achieving 99.05% accuracy on the test set!

# Overlapping, Multi-Digit MNIST
In this problem, we are going to go beyond the basic MNIST. We will train a few neural networks to solve the problem of hand-written digit recognition using a multi-digit version of MNIST.

## Fully connected network
*Complete the code **main** in **mlp.py** to build a fully-connected model with a single hidden layer with 64 units. For this, you need to make use of Linear layers in PyTorch; we provide you with an implementation of `Flatten`, which maps a higher dimensional tensor into an $N x d$ one, where $N$ is the number of samples in your batch and $d$ is the length of the flattened dimension (if your tensor is $N x h x w$, the flattened dimension is  $d=(h \cdot w)$ ). Hint: Note that your model must have two outputs (corresponding to the first and second digits) to be compatible with the data.*

In [137]:
class MLP(nn.Module):

    def __init__(self, input_dimension):
        super(MLP, self).__init__()
        self.flatten = Flatten()
        self.fc1 = nn.Linear(input_dimension, 64)
        self.fc2 = nn.Linear(64, 20) # 20 output neurons (10 digits x 2)

    def forward(self, x):
        xf = self.flatten(x)
        out = self.fc1(xf)
        out = self.fc2(out)
        out_first_digit = out[:, 0:10]
        out_second_digit = out[:, 10:]

        return out_first_digit, out_second_digit
    
    def main():
        X_train, y_train, X_test, y_test = U.get_data(path_to_data_dir, use_mini_dataset)

        # Split into train and dev
        dev_split_index = int(9 * len(X_train) / 10)
        X_dev = X_train[dev_split_index:]
        y_dev = [y_train[0][dev_split_index:], y_train[1][dev_split_index:]]
        X_train = X_train[:dev_split_index]
        y_train = [y_train[0][:dev_split_index], y_train[1][:dev_split_index]]

        permutation = np.array([i for i in range(len(X_train))])
        np.random.shuffle(permutation)
        X_train = [X_train[i] for i in permutation]
        y_train = [[y_train[0][i] for i in permutation], [y_train[1][i] for i in permutation]]

        # Split dataset into batches
        train_batches = batchify_data(X_train, y_train, batch_size)
        dev_batches = batchify_data(X_dev, y_dev, batch_size)
        test_batches = batchify_data(X_test, y_test, batch_size)

        # Load model
        input_dimension = img_rows * img_cols
        model = MLP(input_dimension) # TODO add proper layers to MLP class above

        # Train
        train_model(train_batches, dev_batches, model)

        ## Evaluate the model on test data
        loss, acc = run_epoch(test_batches, model.eval(), None)
        print('Test loss1: {:.6f}  accuracy1: {:.6f}  loss2: {:.6f}   accuracy2: {:.6f}'.format(loss[0], acc[0], loss[1], acc[1]))

In [171]:
%run -i mlp

-------------
Epoch 1:



100%|███████████████████████████████████████████████████████████████████████████████| 562/562 [00:00<00:00, 638.17it/s]


Train | loss1: 0.776068  accuracy1: 0.792538 | loss2: 0.798555  accuracy2: 0.777441


100%|████████████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 1635.91it/s]


Valid | loss1: 0.430175  accuracy1: 0.878780 | loss2: 0.457375  accuracy2: 0.860887
-------------
Epoch 2:



...

Train | loss1: 0.260008  accuracy1: 0.924766 | loss2: 0.285256  accuracy2: 0.916315


100%|████████████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 1593.96it/s]


Valid | loss1: 0.368430  accuracy1: 0.892389 | loss2: 0.353787  accuracy2: 0.898185


100%|████████████████████████████████████████████████████████████████████████████████| 62/62 [00:00<00:00, 1554.12it/s]


-------------
Test loss1: 0.400817  accuracy1: 0.892389  loss2: 0.374367   accuracy2: 0.893901


This model has first and second digit accuracies of 89.24% and 89.39%, respectively. Runtime was 38.6 seconds.

## Convolutional Model

*Complete the code **main** in **conv.py** to build a convolutional model. For this, you need to make use of `Conv2D` layers and `MaxPool2d` layers (and perhaps Dropout) in PyTorch. Make sure that the last layer of the neural network is a fully connected (Linear) layer.*

![CNN Architecture](CNN-example-block-diagram.jpg)

The CNN architecture is diagrammed above. It is modelled after the same architecture as [this tutorial](https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/), but modified to accommodate the overlapping digits as well as to use the helper functions provided by MIT.

In [ ]:
class CNN(nn.Module):

    def __init__(self, input_dimension):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=(5, 7), stride=1, padding=(2,3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 4), stride=(2, 3))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(5, 7), stride=1, padding=(2,3)),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=(2, 3), stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(7 * 10 * 64, 1000)
        self.fc2 = nn.Linear(1000, 20)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        
        out_first_digit = out[:, 0:10]
        out_second_digit = out[:, 10:]

        return out_first_digit, out_second_digit

In [174]:
%run -i conv

-------------
Epoch 1:



100%|████████████████████████████████████████████████████████████████████████████████| 562/562 [01:28<00:00,  6.35it/s]


Train | loss1: 0.586273  accuracy1: 0.807245 | loss2: 0.645280  accuracy2: 0.781611


100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [00:02<00:00, 21.91it/s]


Valid | loss1: 0.181297  accuracy1: 0.942036 | loss2: 0.191528  accuracy2: 0.942036
-------------
Epoch 2:



...

Train | loss1: 0.017875  accuracy1: 0.993856 | loss2: 0.025823  accuracy2: 0.990853


100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [00:02<00:00, 22.04it/s]


Valid | loss1: 0.038975  accuracy1: 0.987903 | loss2: 0.034628  accuracy2: 0.988155


100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [00:02<00:00, 22.29it/s]


Test loss1: 0.049754  accuracy1: 0.986895  loss2: 0.048650   accuracy2: 0.983619


This was a fairly complex model, so the runtime without GPU acceleration was very long (45 minutes 59 seconds). However, the model performs very well at classifying both overlapped digits, with a test accuracy of 98.69% for the first digit and 98.36% for the second digit!

# Conclusions and What's Next
In this project, we manually implemented a basic 3 layer neural network. After successfull testing, we used the PyTorch framework to more easily build neural networks.

We began with the most basic form of a deep neural network, which consisted of fully connected layers of neurons. We attempted to improve accuracy by performing a "mini" grid search over hyperparameters, changing batch size, learning rate, momentum, and the hidden layer activation function. Adjusting the batch size resulted in the highest validation and test accuracy with 0.940020 and	0.931490, respectively, compared to the baseline network's 0.932487	and 0.920472.

We then changed the network architecture by increasing the number of nodes in the hidden layer from 10 to 128. This increased the baseline accuracies to 0.977941 and 0.977163. After performing the same mini grid search, we found that batch size no longer provided the best model, but the LeakyReLU activation, with 0.978108 and	0.977664 validation/test accuracies.

As the last assignment for the single digit MNIST classification, we built a convolutional neural network using PyTorch, which achieved a test accuracy of 0.990485.

Finally, we expanded our CNN to tackle the overlapping, multi-digit MNIST problem. This problem showcased versatility of deep neural networks - simply by changing the output layer, we were able to train the network to a different problem. 

We began by building a fully connected network to solve the problem. This model had first- and second- digit accuracies of 89.24% and 89.39% respectively, and a runtime of only 38.6 seconds.

We then applied a CNN to this problem which was much more complex - taking 46 minutes to complete 30 epochs. However, it performed remarkably well, achieving a test accuracy of 98.69% and 98.36%.

For future work, we will implement the same models with GPU acceleration.